# vnstock trading tools demo
Mặc định phân tích VNINDEX 1 năm gần nhất. Có thể thay `symbol`, `start_date`, `end_date`.

In [ ]:
from datetime import datetime, timedelta
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from vnstock import stock_historical_data

symbol = 'VNINDEX'
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
print(f'Đang tải {symbol} từ {start_date} đến {end_date}...')
df = stock_historical_data(symbol, start_date, end_date, resolution='1D')
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

In [ ]:
def ema(s, span): return s.ewm(span=span, adjust=False).mean()
def rsi(s, length=14):
    delta = s.diff()
    up, down = delta.clip(lower=0), -delta.clip(upper=0)
    ma_up = up.ewm(alpha=1/length, adjust=False).mean()
    ma_down = down.ewm(alpha=1/length, adjust=False).mean()
    rs = ma_up / ma_down
    return 100 - (100 / (1 + rs))

df['EMA9'] = ema(df['close'], 9)
df['EMA26'] = ema(df['close'], 26)
df['EMA100'] = ema(df['close'], 100)
df['EMA200'] = ema(df['close'], 200)
df['RSI9'] = rsi(df['close'], 9)
df['RSI45'] = rsi(df['close'], 45)
df.tail()

In [ ]:
# Đánh dấu giao cắt EMA
df['cross_up'] = (df['EMA9'] > df['EMA26']) & (df['EMA9'].shift(1) <= df['EMA26'].shift(1))
df['cross_dn'] = (df['EMA9'] < df['EMA26']) & (df['EMA9'].shift(1) >= df['EMA26'].shift(1))

# Plot
plt.rcParams.update({'figure.figsize': (14,10), 'font.size': 12})
fig, axes = plt.subplots(3, 1, sharex=True, gridspec_kw={'height_ratios': [3,1,1]})
axes[0].plot(df.index, df['close'], color='black', label='Close')
axes[0].plot(df.index, df['EMA9'], color='blue', label='EMA9')
axes[0].plot(df.index, df['EMA26'], color='orange', label='EMA26')
axes[0].plot(df.index, df['EMA100'], color='purple', label='EMA100')
axes[0].plot(df.index, df['EMA200'], color='red', label='EMA200')
axes[0].scatter(df.index[df['cross_up']], df['close'][df['cross_up']], color='green', s=80, marker='^')
axes[0].scatter(df.index[df['cross_dn']], df['close'][df['cross_dn']], color='red', s=80, marker='v')
axes[0].legend(); axes[0].set_title(symbol)

axes[1].bar(df.index, df['volume'], color='gray')
axes[1].set_ylabel('Volume')

axes[2].plot(df.index, df['RSI9'], label='RSI9', color='blue')
axes[2].plot(df.index, df['RSI45'], label='RSI45', color='green')
axes[2].axhline(70, color='red', ls='--'); axes[2].axhline(30, color='green', ls='--')
axes[2].legend(); axes[2].set_ylabel('RSI')

plt.tight_layout(); plt.show()